In [1]:
import time
import pandas as pd
import numpy as np
import openml
import warnings
import math
import logging

# Imports do scikit-learn e auto-sklearn
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

/opt/conda/envs/askl_env/lib/python3.9/site-packages/sklearn/utils/fixes.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version  # type: ignore


In [2]:
# Configurações
SEED = 42
warnings.filterwarnings("ignore", category=FutureWarning)

def carregar_base_openml(openml_id):
    """Carrega um dataset do OpenML e prepara os tipos de dados."""
    dataset = openml.datasets.get_dataset(openml_id, download_data=True, download_qualities=True, download_features_meta_data=True)
    X, y, _, _ = dataset.get_data(dataset_format="dataframe", target=dataset.default_target_attribute)
    if isinstance(y, pd.Series):
        y = y.values.ravel()
    return X, y

In [10]:
def run_askl_experiment(openml_id):
    """
    Executa o fluxo completo de avaliação para o auto-sklearn em um dataset.
    """
    print(f"--- Processando Dataset ID: {openml_id} com auto-sklearn ---")
    
    # Orçamento de tempo em segundos. Use 3600 para os resultados finais.
    AUTOML_TIME_BUDGET = 3600  # 10 minutos para um teste rápido

    # 1. Carregamento e pré-processamento
    X_df, y_orig = carregar_base_openml(openml_id)
    
    X_train_df, X_test_df, y_train_orig, y_test_orig = train_test_split(
        X_df, y_orig, test_size=0.3, random_state=SEED, stratify=(y_orig if len(np.unique(y_orig)) > 1 else None)
    )

    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_orig)
    y_test_enc = le.transform(y_test_orig)

    # Imputação de NaNs numéricos
    num_cols = X_train_df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0 and X_train_df[num_cols].isnull().sum().sum() > 0:
        imp = SimpleImputer(strategy='median')
        X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
        X_test_df.loc[:, num_cols] = imp.transform(X_test_df[num_cols])

    # Imputação de NaNs categóricos (convertendo para string)
    cat_cols = X_train_df.select_dtypes(include=['category', 'object']).columns
    if len(cat_cols) > 0:
        for col in cat_cols:
            X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
            X_test_df.loc[:, col] = X_test_df[col].astype(str).fillna("__MISSING__")
            
    # 2. Treinamento com auto-sklearn
    t0_total = time.time()

    automl = AutoSklearn2Classifier(
        time_left_for_this_task=AUTOML_TIME_BUDGET,
        n_jobs=-1,                
        seed=SEED,
        memory_limit=None
    )
    
    # O auto-sklearn lida bem com DataFrames pré-processados
    automl.fit(X_train_df, y_train_enc)
    
    # 3. Predição e Métricas
    y_proba = automl.predict_proba(X_test_df)
    y_pred = automl.predict(X_test_df)
    total_time = time.time() - t0_total

    n_classes = len(le.classes_)
    if n_classes == 2:
        auc_ovo = roc_auc_score(y_test_enc, y_proba[:, 1])
    else:
        auc_ovo = roc_auc_score(y_test_enc, y_proba, multi_class='ovo')

    acc = accuracy_score(y_test_enc, y_pred)
    ce = log_loss(y_test_enc, y_proba, labels=range(n_classes))
    
    # Extrai o melhor score de validação cruzada dos resultados internos
    best_score_cv = np.max(automl.cv_results_['mean_test_score'])


    print(f"Resultados para {openml_id}: AUC={auc_ovo:.4f}, ACC={acc:.4f}, Time={total_time:.2f}s")
    
    return {
        'openml_id': openml_id, 'modelo': 'autosklearn', 'total_time_sec': total_time,
        'mean_auc_ovo': auc_ovo, 'mean_accuracy': acc, 'mean_cross_entropy': ce,
        'best_model_info': 'Ensemble', # O melhor modelo é um ensemble complexo
        'best_score_cv': best_score_cv
    }

In [4]:
logging.getLogger("autosklearn").setLevel(logging.INFO)

MACHINE_ID = 1
TOTAL_MACHINES = 4

cc18_ids = [
    11, 15, 18, 23, 29, 31, 37, 50, 54, 188,
    307, 458, 469, 1049, 1050, 1063, 1068, 1462, 1464, 1468,
    1480, 1494, 1501, 1510, 6332, 23381, 40966, 40975, 40982, 40994
]

chunk_size = math.ceil(len(cc18_ids) / TOTAL_MACHINES)
start_index = (MACHINE_ID - 1) * chunk_size
end_index = start_index + chunk_size
ids_to_process = cc18_ids[start_index:end_index]

print(f"--- MÁQUINA {MACHINE_ID}/{TOTAL_MACHINES} ---")
print(f"Processando os seguintes datasets: {ids_to_process}")
    
todos_resultados_askl = []

for oid in ids_to_process:
    try:
        res = run_askl_experiment(oid)
        todos_resultados_askl.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

if todos_resultados_askl:
    df_askl_results = pd.DataFrame(todos_resultados_askl)
    print("\n\n--- RESULTADOS FINAIS DO AUTO-SKLEARN ---")
    print(df_askl_results)
    df_askl_results.to_csv("resultados_autosklearn.csv", index=False)
    print("\nArquivo 'resultados_autosklearn.csv' salvo com sucesso!")
else:
    print("\nNenhuma tarefa foi concluída com sucesso.")

--- MÁQUINA 1/4 ---
Processando os seguintes datasets: [11, 15, 18, 23, 29, 31, 37, 50]
--- Processando Dataset ID: 11 com auto-sklearn ---


/opt/conda/envs/askl_env/lib/python3.9/site-packages/sklearn/utils/fixes.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version  # type: ignore


[WARNING] [2025-06-15 01:03:25,361:Client-AutoML(42):86e2afc2-4984-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
Resultados para 11: AUC=0.9962, ACC=0.9787, Time=3606.25s
--- Processando Dataset ID: 15 com auto-sklearn ---


/var/tmp/ipykernel_77942/1891648276.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
/var/tmp/ipykernel_77942/1891648276.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_df.loc[:, num_cols] = imp.transform(X_test_df[num_cols])


[WARNING] [2025-06-15 02:03:32,807:Client-AutoML(42):ec725d04-498c-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2025-06-15 02:03:35,385:Client-EnsembleBuilder] No runs were available to build an ensemble from


...

Traceback (most recent call last):
  File "/var/tmp/ipykernel_77942/3206329540.py", line 24, in <module>
    res = run_askl_experiment(oid)
  File "/var/tmp/ipykernel_77942/1891648276.py", line 46, in run_askl_experiment
    automl.fit(X_train_df, y_train_enc)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/experimental/askl2.py", line 540, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 1448, in fit
    super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 540, in fit
    self.automl_.fit(load_models=self.load_models, **kwargs)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 2304, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 962, in fit
    raise e
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autoskle

[WARNING] [2025-06-15 04:04:42,285:Client-AutoML(42):d8069420-499d-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2025-06-15 04:04:44,962:Client-EnsembleBuilder] No runs were available to build an ensemble from
Resultados para 37: AUC=0.8396, ACC=0.7489, Time=3611.74s
--- Processando Dataset ID: 50 com auto-sklearn ---


/var/tmp/ipykernel_77942/1891648276.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/1891648276.py:32: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/1891648276.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

[WARNING] [2025-06-15 05:04:45,803:Client-AutoML(42):40e2b12a-49a6-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[ERROR] [2025-06-15 05:04:45,907:Client-AutoML(42):40e2b12a-49a6-11f0-b076-42010a800005] list index out of range
Traceback (most recent call last):
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/smbo.py", line 552, in run_smbo
    smac.optimize()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 211, in run
    self.start()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 174, in start
    self.initial_design_configs = self

Traceback (most recent call last):
  File "/var/tmp/ipykernel_77942/3206329540.py", line 24, in <module>
    res = run_askl_experiment(oid)
  File "/var/tmp/ipykernel_77942/1891648276.py", line 46, in run_askl_experiment
    automl.fit(X_train_df, y_train_enc)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/experimental/askl2.py", line 540, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 1448, in fit
    super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 540, in fit
    self.automl_.fit(load_models=self.load_models, **kwargs)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 2304, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 962, in fit
    raise e
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autoskle

In [8]:
from autosklearn.classification import AutoSklearnClassifier

def run_askl_experiment_optional(openml_id):
    """
    Executa o fluxo completo de avaliação para o auto-sklearn em um dataset.
    """
    print(f"--- Processando Dataset ID: {openml_id} com auto-sklearn ---")
    
    # Orçamento de tempo em segundos. Use 3600 para os resultados finais.
    AUTOML_TIME_BUDGET = 3600  # 10 minutos para um teste rápido

    # 1. Carregamento e pré-processamento
    X_df, y_orig = carregar_base_openml(openml_id)
    
    X_train_df, X_test_df, y_train_orig, y_test_orig = train_test_split(
        X_df, y_orig, test_size=0.3, random_state=SEED, stratify=(y_orig if len(np.unique(y_orig)) > 1 else None)
    )

    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_orig)
    y_test_enc = le.transform(y_test_orig)

    # Imputação de NaNs numéricos
    num_cols = X_train_df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0 and X_train_df[num_cols].isnull().sum().sum() > 0:
        imp = SimpleImputer(strategy='median')
        X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
        X_test_df.loc[:, num_cols] = imp.transform(X_test_df[num_cols])

    # Imputação de NaNs categóricos (convertendo para string)
    cat_cols = X_train_df.select_dtypes(include=['category', 'object']).columns
    if len(cat_cols) > 0:
        for col in cat_cols:
            X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
            X_test_df.loc[:, col] = X_test_df[col].astype(str).fillna("__MISSING__")
            
    # 2. Treinamento com auto-sklearn
    t0_total = time.time()

    automl = AutoSklearnClassifier(
        time_left_for_this_task=AUTOML_TIME_BUDGET,
        n_jobs=-1,                
        seed=SEED,
        memory_limit=None
        #initial_configurations_via_metalearning=0
    )
    
    # O auto-sklearn lida bem com DataFrames pré-processados
    automl.fit(X_train_df, y_train_enc)
    
    # 3. Predição e Métricas
    y_proba = automl.predict_proba(X_test_df)
    y_pred = automl.predict(X_test_df)
    total_time = time.time() - t0_total

    n_classes = len(le.classes_)
    if n_classes == 2:
        auc_ovo = roc_auc_score(y_test_enc, y_proba[:, 1])
    else:
        auc_ovo = roc_auc_score(y_test_enc, y_proba, multi_class='ovo')

    acc = accuracy_score(y_test_enc, y_pred)
    ce = log_loss(y_test_enc, y_proba, labels=range(n_classes))
    
    # Extrai o melhor score de validação cruzada dos resultados internos
    best_score_cv = np.max(automl.cv_results_['mean_test_score'])


    print(f"Resultados para {openml_id}: AUC={auc_ovo:.4f}, ACC={acc:.4f}, Time={total_time:.2f}s")
    
    return {
        'openml_id': openml_id, 'modelo': 'autosklearn', 'total_time_sec': total_time,
        'mean_auc_ovo': auc_ovo, 'mean_accuracy': acc, 'mean_cross_entropy': ce,
        'best_model_info': 'Ensemble', # O melhor modelo é um ensemble complexo
        'best_score_cv': best_score_cv
    }

In [9]:
ids_to_process = [23, 29, 31, 50]

print(f"Processando os seguintes datasets: {ids_to_process}")
    
todos_resultados_askl = []

for oid in ids_to_process:
    try:
        res = run_askl_experiment_optional(oid)
        todos_resultados_askl.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

if todos_resultados_askl:
    df_askl_results = pd.DataFrame(todos_resultados_askl)
    print("\n\n--- RESULTADOS FINAIS DO AUTO-SKLEARN ---")
    print(df_askl_results)
    df_askl_results.to_csv("resultados_autosklearn2.csv", index=False)
    print("\nArquivo 'resultados_autosklearn.csv' salvo com sucesso!")
else:
    print("\nNenhuma tarefa foi concluída com sucesso.")

Processando os seguintes datasets: [23, 29, 31, 50]
--- Processando Dataset ID: 23 com auto-sklearn ---


/var/tmp/ipykernel_77942/2533747859.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/2533747859.py:34: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/2533747859.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

[WARNING] [2025-06-15 07:42:55,983:Client-AutoML(42):591d0257-49bc-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


Process pynisher function call:
Traceback (most recent call last):
  File "/opt/conda/envs/askl_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/askl_env/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/pynisher/limit_function_call.py", line 133, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/smbo.py", line 160, in _calculate_metafeatures_encoded
    result = calculate_all_metafeatures_encoded_labels(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/metalearning/metafeatures/metafeatures.py", line 1115, in calculate_all_metafeatures_encoded_labels
    return calculate_all_metafeatures(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/metalearning/metafeatures/metafeatures

[WARNING] [2025-06-15 07:42:58,264:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:58,388:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:58,582:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:58,689:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:58,820:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:59,565:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:59,721:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:42:59,865:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:43:00,184:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:43:

...

[WARNING] [2025-06-15 09:43:00,608:Client-AutoML(42):1a90abb4-49cd-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2025-06-15 09:43:04,739:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 09:43:04,880:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 09:43:05,225:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 09:43:05,376:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 09:43:05,966:Client-EnsembleBuilder] No runs were available to build an ensemble from
Resultados para 31: AUC=0.7068, ACC=0.7267, Time=3612.17s
--- Processando Dataset ID: 50 com auto-sklearn ---


/var/tmp/ipykernel_77942/2533747859.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/2533747859.py:34: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/2533747859.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

[WARNING] [2025-06-15 10:43:12,002:Client-AutoML(42):83b3dc5c-49d5-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


Process pynisher function call:
Traceback (most recent call last):
  File "/opt/conda/envs/askl_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/askl_env/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/pynisher/limit_function_call.py", line 133, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/smbo.py", line 160, in _calculate_metafeatures_encoded
    result = calculate_all_metafeatures_encoded_labels(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/metalearning/metafeatures/metafeatures.py", line 1115, in calculate_all_metafeatures_encoded_labels
    return calculate_all_metafeatures(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/metalearning/metafeatures/metafeatures

[WARNING] [2025-06-15 10:43:14,303:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:14,548:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:14,753:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:14,907:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:15,601:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:15,710:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:16,008:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:16,243:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:16,561:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 10:43:

In [12]:
ids_to_process = [1480]

print(f"Processando os seguintes datasets: {ids_to_process}")
    
todos_resultados_askl = []

for oid in ids_to_process:
    try:
        res = run_askl_experiment_optional(oid)
        todos_resultados_askl.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

if todos_resultados_askl:
    df_askl_results = pd.DataFrame(todos_resultados_askl)
    print("\n\n--- RESULTADOS FINAIS DO AUTO-SKLEARN ---")
    print(df_askl_results)
    df_askl_results.to_csv("resultados_autosklearn_1480.csv", index=False)
    print("\nArquivo 'resultados_autosklearn.csv' salvo com sucesso!")
else:
    print("\nNenhuma tarefa foi concluída com sucesso.")

Processando os seguintes datasets: [1480]
--- Processando Dataset ID: 1480 com auto-sklearn ---


/var/tmp/ipykernel_77942/2533747859.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/2533747859.py:34: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_77942/2533747859.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

[WARNING] [2025-06-15 11:49:03,923:Client-AutoML(42):b6e11dd8-49de-11f0-b076-42010a800005] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


/opt/conda/envs/askl_env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37461 instead
  warnings.warn(


[WARNING] [2025-06-15 11:49:06,388:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 11:49:06,514:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 11:49:06,634:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 11:49:07,187:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2025-06-15 11:49:07,433:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
Resultados para 1480: AUC=0.6630, ACC=0.6686, Time=3614.90s


--- RESULTADOS FINAIS DO AUTO-SKLEARN ---
   openml_id       modelo  total_time_sec  mean_auc_ovo  mean_accuracy  \
0       1480  autosklearn     3614.902052       0.66304       0.668571   

   mean_cross_entropy best_model_info  best_score_cv  
0            0.555246        Ensembl